In [275]:
import pandas as pd
import numpy as np
import glob
import re

Loading Files

In [276]:
#Loads all csv files and merges them into one dataframe.
all_csv = glob.glob("*.csv")
cars = pd.DataFrame()
for csv in all_csv:
    car = pd.read_csv(csv,index_col=0)
    cars = pd.concat([cars, car], axis=0, ignore_index=True)

There are currently over 29000 rows in the dataset. However, this is expected to decrease after the data has been cleaned.
There are a number of null values across most columns in the dataset. These will be removed.
Many columns have the wrong data type. For example, the year column is currently an object, but should be an integer.

In [277]:

cars.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29370 entries, 0 to 29369
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   name                29328 non-null  object
 1   year                27326 non-null  object
 2   owners              17756 non-null  object
 3   mileage             27316 non-null  object
 4   litres              27272 non-null  object
 5   BHP                 27265 non-null  object
 6   transmission        27239 non-null  object
 7   fuel                26997 non-null  object
 8   price               27328 non-null  object
 9   write_off_category  1119 non-null   object
 10  vehicle_type        27326 non-null  object
 11  seller_name         27799 non-null  object
 12  location            25049 non-null  object
dtypes: object(13)
memory usage: 2.9+ MB


In [278]:
cars.head()


,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location
0,KIA Sportage,2018,NaN,"23,600 miles",1.6L,176PS,Manual,Petrol,"£16,990",NaN,SUV,Belfast Audi - See all 68 cars,Dealer locationBelfast
1,Vauxhall Astra,2007 (57 reg),Part service history,"119,000 miles",1.9L,118BHP,Automatic,Diesel,£995,NaN,Hatchback,Burch Autos - See all 30 cars,Dealer locationNewtownards
2,Citroen DS3,2012 (12 reg),NaN,"75,995 miles",1.6L,156BHP,Manual,Petrol,"£1,493",NaN,Hatchback,Auto Sales NI - See all cars,Dealer locationNewtownabbey
3,Audi A3,2007 (57 reg),NaN,"98,968 miles",2.0L,168BHP,Manual,Diesel,"£1,999",NaN,Hatchback,Clearly Cars - See all 17 cars,Dealer locationBelfast
4,Suzuki Swift,2007 (07 reg),4 owners,"67,200 miles",1.5L,101BHP,Manual,Petrol,"£1,295",NaN,Hatchback,Charles Hurst Usedirect Budget Choice - See al...,Dealer locationBelfast


In [279]:
cars.duplicated().sum()
# cars.duplicated(subset=["price", "owners", "mileage", "name", "litres", "fuel", "transmission", "write_off_category", "location", "seller_name", "BHP"]).sum()


18859

In [280]:
# cars = cars.reset_index(drop=True)


Due to the way the data was obtained there are several duplicate rows, the first step is to remove all the repeated entries.

In [281]:
#List duplicates
# len(cars.duplicated())
len(cars[cars.duplicated()])


18859

In [282]:
cars.drop_duplicates(inplace=True)
cars.shape


(10511, 13)

The next step is to remove all the entries with missing values. The missing values of some columns may require further inspection, so removal will be carried out on a column by column basis.

Name Column

In [283]:
(cars.name).isnull().sum()
cars = cars[cars.name.notnull()]



The web scraper also collected data on advertised cars which usually did not include much of the required information and may explain a large portion of the null values.

Price Column

In [284]:
# cars[cars.price.isnull()].size
cars[cars.price.isnull()]

,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location
1313,Vauxhall Corsa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN
1327,SEAT Arona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN
1341,Volkswagen T-Cross,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN
1355,Vauxhall Astra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN
1369,SKODA Octavia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615,SKODA Karoq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN
2629,Renault Captur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN
2657,SEAT Ibiza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN
5499,Renault Clio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,See all lease deals,NaN


In [285]:
cars = cars[cars.price.notnull()]
cars.shape

(10417, 13)

In [ ]:
#All values in the price column have a £ (Great British Pound (GBP) ) sign at the start of the string
#The £ sign needs to be removed in order to convert the values to a float or integer
cars.price.str[0].unique()
cars.price = cars.price.str.replace("£", "")
cars.price = cars.price.str.replace(",", "")
cars.price = cars.price.astype(float)


Year Column

In [292]:
# cars.reset_index(drop=True, inplace=True)

cars[cars.year.isnull()].size

26

In [293]:
cars = cars[cars.year.notnull()]

cars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10415 entries, 0 to 29366
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   name                10415 non-null  object
 1   year                10415 non-null  object
 2   owners              6699 non-null   object
 3   mileage             10407 non-null  object
 4   litres              10406 non-null  object
 5   BHP                 10402 non-null  object
 6   transmission        10384 non-null  object
 7   fuel                10318 non-null  object
 8   price               10415 non-null  object
 9   write_off_category  383 non-null    object
 10  vehicle_type        10415 non-null  object
 11  seller_name         9685 non-null   object
 12  location            9076 non-null   object
dtypes: object(13)
memory usage: 1.1+ MB


Some columns appear to have information from the adjacent column on the website. This is likely due to the slightly different format of the advertisments.

In [ ]:
cars["year"].unique()

array(['2018', '2007 (57 reg)', '2012 (12 reg)', '2007 (07 reg)',
       '2009 (59 reg)', '2006 (06 reg)', '2011 (61 reg)', '2013 (63 reg)',
       '2005', '2020', '2008', '2011', '2017', '2009', '2015', '2016',
       '2010', '2012', '2013', '2014', '2007', '1995', '1998', '2019',
       '1999', '2006', '2021', '2004', '2022', 'Saloon', '2023',
       'Hatchback', 'SUV', '1978', '1972', '2016 (16 reg)',
       '2003 (03 reg)', '2012 (61 reg)', '2010 (59 reg)', '2014 (14 reg)',
       '2018 (68 reg)', '2015 (65 reg)', '2010 (60 reg)', '2013 (13 reg)',
       '2012 (62 reg)', '2010 (10 reg)', '2008 (08 reg)', '2001 (51 reg)',
       '2002 (02 reg)', '2004 (54 reg)', '2019 (69 reg)', '2008 (58 reg)',
       '2011 (11 reg)', '2006 (56 reg)', '2004 (04 reg)', '2005 (54 reg)',
       '2017 (17 reg)', '2009 (09 reg)', '2019 (19 reg)', '2006 (55 reg)',
       '2003 (53 reg)', '2000 (W reg)', '2008 (57 reg)', '2013 (62 reg)',
       '1998 (R reg)', '2005 (05 reg)', '1972 (K reg)', '2009 (58 re

In [ ]:
cars[cars["year"]=="SUV"].size
# cars.shape

60

In [288]:
print(cars[cars["year"]=="Hatchback"])   

                name       year owners mileage  litres     BHP transmission  \
764  Volkswagen Polo  Hatchback    NaN    1.0L  Manual  Petrol          NaN   
840      Ford Fiesta  Hatchback    NaN  Manual  Petrol     NaN          NaN   

    fuel  price write_off_category  vehicle_type  \
764  NaN  14490                NaN  31,313 miles   
840  NaN  11995                NaN  29,552 miles   

                                    seller_name                     location  
764  Agnew Volkswagen Mallusk - See all 67 cars   Dealer locationNewtonabbey  
840       Ballyrobert Mallusk - See all 99 cars  Dealer locationNewtownabbey  


In [289]:
cars[cars["year"]=="Saloon"]

,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location
249,Mercedes-Benz S Class,Saloon,NaN,3.0L,Automatic,Diesel,1 owner,NaN,18250,NaN,"92,000 miles",JGM GROUP - See all 28 cars,Dealer locationBelfast


In [287]:
cars.head()

,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location
0,KIA Sportage,2018,NaN,"23,600 miles",1.6L,176PS,Manual,Petrol,16990,NaN,SUV,Belfast Audi - See all 68 cars,Dealer locationBelfast
1,Vauxhall Astra,2007 (57 reg),Part service history,"119,000 miles",1.9L,118BHP,Automatic,Diesel,995,NaN,Hatchback,Burch Autos - See all 30 cars,Dealer locationNewtownards
2,Citroen DS3,2012 (12 reg),NaN,"75,995 miles",1.6L,156BHP,Manual,Petrol,1493,NaN,Hatchback,Auto Sales NI - See all cars,Dealer locationNewtownabbey
3,Audi A3,2007 (57 reg),NaN,"98,968 miles",2.0L,168BHP,Manual,Diesel,1999,NaN,Hatchback,Clearly Cars - See all 17 cars,Dealer locationBelfast
4,Suzuki Swift,2007 (07 reg),4 owners,"67,200 miles",1.5L,101BHP,Manual,Petrol,1295,NaN,Hatchback,Charles Hurst Usedirect Budget Choice - See al...,Dealer locationBelfast


In [ ]:
#Many of the null year values also have null values in other columns


When the year is missing, the registration plate information can be used to verify the correct year for each row. https://www.theaa.com/car-buying/number-plates

In [295]:
cars[cars.year.str.contains('54')]
cars.loc[cars.year.str.contains('54'), "year"] = "2004"
cars.loc[cars.year.str.contains('66'), "year"] = "2016"
cars.loc[cars.year.str.contains('12'), "year"] = "2012"
cars[cars.year.isnull()].size


0

In [296]:
cars.year.unique()

array(['2018', '2007 (57 reg)', '2012', '2007 (07 reg)', '2009 (59 reg)',
       '2006 (06 reg)', '2011 (61 reg)', '2013 (63 reg)', '2005', '2020',
       '2008', '2011', '2017', '2009', '2015', '2016', '2010', '2013',
       '2014', '2007', '1995', '1998', '2019', '1999', '2006', '2021',
       '2004', '2022', 'Saloon', '2023', 'Hatchback', 'SUV', '1978',
       '1972', '2016 (16 reg)', '2003 (03 reg)', '2010 (59 reg)',
       '2014 (14 reg)', '2018 (68 reg)', '2015 (65 reg)', '2010 (60 reg)',
       '2013 (13 reg)', '2010 (10 reg)', '2008 (08 reg)', '2001 (51 reg)',
       '2002 (02 reg)', '2019 (69 reg)', '2008 (58 reg)', '2011 (11 reg)',
       '2006 (56 reg)', '2004 (04 reg)', '2017 (17 reg)', '2009 (09 reg)',
       '2019 (19 reg)', '2006 (55 reg)', '2003 (53 reg)', '2000 (W reg)',
       '2008 (57 reg)', '2013 (62 reg)', '1998 (R reg)', '2005 (05 reg)',
       '1972 (K reg)', '2009 (58 reg)', '1999 (V reg)', '2000 (X reg)',
       '2021 (71 reg)', '2002 (51 reg)', '2001 (Y reg)'

In [ ]:
#Starting with the year column 
#All years should be exactly 4 digits
def get_year(x):
    x = str(x)
    if len(x) <=4:
        return x
    else:
        return x[:4]

#There could be some useful information regarding number plates.            (you never know)
#If the 6th value in the string is a number then return the 6th and 7th value.
#Retrieving the license plate of the car

def get_plate(x):
    x = str(x)
    return x[6:8]
cars['plate'] = cars["year"].apply(get_plate)
cars['year'] = cars["year"].apply(get_year)




In [300]:
cars["year"].unique()

array(['2018', '2007', '2012', '2009', '2006', '2011', '2013', '2005',
       '2020', '2008', '2017', '2015', '2016', '2010', '2014', '1995',
       '1998', '2019', '1999', '2021', '2004', '2022', 'Salo', '2023',
       'Hatc', 'SUV', '1978', '1972', '2003', '2001', '2002', '2000',
       '1991', '1994', '1989', '1997', '1996', '1993', '1987', '1988',
       '1986', '1968', '1992', '1980', '1981', '1975', '1970', '1973',
       '1935', '1984'], dtype=object)

In [301]:
cars = cars[cars["year"].apply(lambda x : x.isnumeric())]


In [305]:
cars.year.unique()


array(['2018', '2007', '2012', '2009', '2006', '2011', '2013', '2005',
       '2020', '2008', '2017', '2015', '2016', '2010', '2014', '1995',
       '1998', '2019', '1999', '2021', '2004', '2022', '2023', '1978',
       '1972', '2003', '2001', '2002', '2000', '1991', '1994', '1989',
       '1997', '1996', '1993', '1987', '1988', '1986', '1968', '1992',
       '1980', '1981', '1975', '1970', '1973', '1935', '1984'],
      dtype=object)

Taking a closer look at the year values, the 1935 value was particularly intresting. After further inspection, it was confirmed to accurate.
https://en.wikipedia.org/wiki/Morris_Minor_(1928)

In [311]:
cars["year"] = cars["year"].astype(int)

In [312]:
cars[cars["year"]==1935]

,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location,plate
20554,Morris Minor,1935,NaN,"55,861 miles",0.9L,Manual,Petrol,6 owners,8750,NaN,Saloon,NaN,NaN,


In [313]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10408 entries, 0 to 29366
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   name                10408 non-null  object
 1   year                10408 non-null  int32 
 2   owners              6699 non-null   object
 3   mileage             10400 non-null  object
 4   litres              10399 non-null  object
 5   BHP                 10396 non-null  object
 6   transmission        10379 non-null  object
 7   fuel                10314 non-null  object
 8   price               10408 non-null  object
 9   write_off_category  383 non-null    object
 10  vehicle_type        10408 non-null  object
 11  seller_name         9678 non-null   object
 12  location            9069 non-null   object
 13  plate               10408 non-null  object
dtypes: int32(1), object(13)
memory usage: 1.2+ MB


Mileage Column

In [314]:
#Removes all non-numeric characters from the mileage column.
cars.mileage
cars["mileage"] = cars["mileage"].apply(lambda x: re.sub("[^0-9]", "", str(x)))


In [315]:
cars["mileage"].isnull().sum()

0

In [316]:
cars[~cars.mileage.str.isnumeric()]

,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location,plate
49,Toyota Aristo,1998,NaN,,NaN,NaN,NaN,NaN,11995,NaN,"80,294 miles",Auto Choice - See all 59 cars,Dealer locationNewtownards,
9711,BMW Alpina B10,2003,NaN,,NaN,NaN,NaN,NaN,5995,NaN,"123,000 miles",Unique Motors - See all 22 cars,Dealer locationCastleford,52
10436,Nissan Elgrand,2017,NaN,,NaN,NaN,NaN,NaN,6000,NaN,"100,000 miles",HSI Automotive - See all 21 cars,Dealer locationBradford,67
13912,Peugeot 208,2020,NaN,,NaN,NaN,NaN,NaN,10100,NaN,Hatchback,Hilton Car Supermarket Milton Keynes - See all...,Dealer locationMilton Keynes,20
14412,Ford Dorchester,2002,NaN,,NaN,NaN,NaN,NaN,2999,NaN,"80,600 miles",Ideal Carz - See all 51 cars,Dealer locationMilton Keynes,52
14637,Toyota Estima,2021,NaN,,NaN,NaN,NaN,NaN,7490,NaN,"88,000 miles",Auto Globe Limited - See all 113 cars,Dealer locationMilton Keynes,21
19580,Hyundai Coupe,2007,NaN,,NaN,NaN,NaN,NaN,1895,NaN,Coupe,Eden Approved Bicester - See all 48 cars,Dealer locationBicester,07
19941,Toyota Estima,2021,NaN,,NaN,NaN,NaN,NaN,7490,NaN,"88,000 miles",Auto Globe Limited - See all 114 cars,NaN,21
22058,Toyota Estima,2021,NaN,,NaN,NaN,NaN,NaN,7490,NaN,"88,000 miles",Auto Globe Limited - See all 118 cars,Dealer locationMilton Keynes,21


In [317]:
cars.loc[cars.mileage=="", "mileage"] = np.nan

In [318]:
cars = cars[cars["mileage"].notnull()]

In [319]:
cars["mileage"] = cars["mileage"].astype(float)

In [320]:
cars["mileage"].isnull().sum()

0

Litres column

In [324]:
# cars[cars.litres.isnull()].size
# print(cars.litres.unique())
cars[cars.litres.str.contains("L", na=False)].size

cars = cars[cars.litres.str.contains("L", na=False)]
# cars.shape
cars.litres.unique()

array(['1.6L', '1.9L', '2.0L', '1.5L', '1.2L', '5.0L', '1.4L', '1.33L',
       '1.8L', '1.0L', '3.0L', '2.1L', '1.1L', '0.9L', '2.4L', '1.7L',
       '1.3L', '2.2L', '3.5L', '1.25L', '2.5L', '4.4L', '5.5L', '3.2L',
       '2.3L', '4.2L', '2.7L', '0.66L', '2.8L', '2.9L', '3.3L', '0.7L',
       '0.6L', '4.0L', '6.0L', '4.6L', '7.7L', '3.7L', '4.3L', '2.6L',
       '6.2L', '0.1L', '3.6L', '3.4L'], dtype=object)

Horsepower column

In [325]:
# cars[cars["BHP"].isnull()]
#BHP null values can safely be removed
cars[cars["BHP"].isnull()].head()
cars = cars[cars["BHP"].notnull()]

In [326]:
#Non-numeric characters from the BHP column need to be removed
cars["BHP"].unique()

array(['176PS', '118BHP', '156BHP', '168BHP', '101BHP', '113BHP',
       '130BHP', '85PS', '110BHP', 'Automatic', '190PS', '67BHP', '143PS',
       '188BHP', '100BHP', '136PS', '110PS', '115PS', '100PS', '116PS',
       '170BHP', '140PS', '123BHP', '75PS', '79BHP', 'Manual', '60PS',
       '197PS', '68BHP', '88BHP', '156PS', '150PS', '98PS', '114BHP',
       '70PS', '122PS', '142PS', '120PS', '326PS', '174BHP', '90BHP',
       '148BHP', '163PS', '132PS', '184PS', '245BHP', '137PS', '68PS',
       '138BHP', '105PS', '125PS', '114PS', '130PS', '125BHP', '83PS',
       '89BHP', '86PS', '91BHP', '313PS', '181BHP', '66PS', '90PS',
       '147PS', '187BHP', '121PS', '150BHP', '111PS', '143BHP', '134BHP',
       '77BHP', '87PS', '120BHP', '145PS', '69BHP', '128BHP', '200PS',
       '109PS', '131PS', '84BHP', '170PS', '276BHP', '72BHP', '160PS',
       '117PS', '147BHP', '69PS', '83BHP', '80BHP', '106PS', '139BHP',
       '136BHP', '60BHP', '197BHP', '59BHP', '180PS', '278BHP', '84PS',
       

PS (Pferdestärke, meaning horse strength in German) and  (Brake Horsepower) are often used interchangeably. 

One PS is roughly 98.6% of a BHP


In [327]:

cars["BHP"].str[-2:].unique()
PS = cars["BHP"].str.contains("PS")
cars.loc[PS,'BHP'] = cars.loc[PS,'BHP'].str[:-2].astype(int) * 0.98632
    

In [328]:
cars["BHP"].str[-2:].unique()

array([nan, 'HP', 'ic', 'al', 'el', 'ol'], dtype=object)

In [329]:
HP = cars["BHP"].str.contains("BHP", na=False)
cars.loc[HP, 'BHP'] = cars.loc[HP, 'BHP'].str.replace("BHP", "").astype(int)

In [330]:

cars[cars["BHP"].str.isnumeric() == False]
cars["BHP"] = cars["BHP"].apply(lambda x: re.sub("[^0-9.]", "", str(x)))

In [332]:
cars.loc[cars.BHP=="", "BHP"] = np.nan
cars["BHP"] = cars["BHP"].astype(float).round()
cars["BHP"]

0        174.0
1        118.0
2        156.0
3        168.0
4        101.0
         ...  
29361    123.0
29362    177.0
29364     68.0
29365    268.0
29366    204.0
Name: BHP, Length: 10304, dtype: float64

In [334]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10304 entries, 0 to 29366
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                10304 non-null  object 
 1   year                10304 non-null  int32  
 2   owners              6695 non-null   object 
 3   mileage             10304 non-null  float64
 4   litres              10304 non-null  object 
 5   BHP                 10228 non-null  float64
 6   transmission        10303 non-null  object 
 7   fuel                10276 non-null  object 
 8   price               10304 non-null  object 
 9   write_off_category  381 non-null    object 
 10  vehicle_type        10304 non-null  object 
 11  seller_name         9591 non-null   object 
 12  location            8989 non-null   object 
 13  plate               10304 non-null  object 
dtypes: float64(2), int32(1), object(11)
memory usage: 1.1+ MB


In [ ]:
cars.head() 

,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location
0,KIA Sportage,2018,NaN,23600.0,1.6L,174.0,Manual,Petrol,"£16,990",NaN,SUV,Belfast Audi - See all 68 cars,Dealer locationBelfast
2,Citroen DS3,2012,NaN,75995.0,1.6L,156.0,Manual,Petrol,"£1,493",NaN,Hatchback,Auto Sales NI - See all cars,Dealer locationNewtownabbey
9,Mercedes-Benz CLS,2005,NaN,87200.0,5.0L,NaN,Petrol,NaN,"£4,995",NaN,Saloon,RLW Cars - See all 4 cars,"Dealer locationBangor, County Down"
13,Audi Q5,2020,NaN,21000.0,2.0L,187.0,Automatic,Diesel,"£34,990",NaN,SUV,Belfast Audi - See all 68 cars,Dealer locationBelfast
14,Ford Fiesta,2008,5 owners,100557.0,1.4L,67.0,Manual,Diesel,"£2,799",NaN,Hatchback,RLW Cars - See all 3 cars,"Dealer locationBangor, County Down"


Vehicle Type Column

In [ ]:
cars.vehicle_type.unique()

array(['SUV', 'Hatchback', 'Saloon', 'Coupe', 'Convertible', 'MPV',
       'Estate', 'Minibus', 'Pickup', 'Combi Van'], dtype=object)

In [335]:
# cars[cars["write_off_category"].isna()]
#Vehicles with no write off category were automatically given null values
#GC stands for good condition
cars["write_off_category"] = cars["write_off_category"].fillna("GC")
cars["write_off_category"].unique()
cars

,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location,plate
0,KIA Sportage,2018,NaN,23600.0,1.6L,174.0,Manual,Petrol,16990,GC,SUV,Belfast Audi - See all 68 cars,Dealer locationBelfast,
1,Vauxhall Astra,2007,Part service history,119000.0,1.9L,118.0,Automatic,Diesel,995,GC,Hatchback,Burch Autos - See all 30 cars,Dealer locationNewtownards,57
2,Citroen DS3,2012,NaN,75995.0,1.6L,156.0,Manual,Petrol,1493,GC,Hatchback,Auto Sales NI - See all cars,Dealer locationNewtownabbey,
3,Audi A3,2007,NaN,98968.0,2.0L,168.0,Manual,Diesel,1999,GC,Hatchback,Clearly Cars - See all 17 cars,Dealer locationBelfast,57
4,Suzuki Swift,2007,4 owners,67200.0,1.5L,101.0,Manual,Petrol,1295,GC,Hatchback,Charles Hurst Usedirect Budget Choice - See al...,Dealer locationBelfast,07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29361,Vauxhall Zafira,2013,3 owners,102500.0,1.7L,123.0,Manual,Diesel,2490,GC,MPV,HCL Prestige Car Supermarket - See all 69 cars,Dealer locationAshford,63
29362,Mercedes-Benz E Class,2003,NaN,143000.0,2.7L,177.0,Automatic,Diesel,1950,GC,Saloon,Auto 4 U - See all 109 cars,Dealer locationGreenford,03
29364,Ford KA,2011,NaN,70000.0,1.2L,68.0,Manual,Petrol,3390,GC,Hatchback,GM Motoring World LTD - See all 93 cars,Dealer locationGreat Missenden,11
29365,Mercedes-Benz C Class,2008,Full service history,112000.0,3.5L,268.0,Automatic,Petrol,4950,GC,Saloon,Autoland - See all 80 cars,Dealer locationGreenford,08


Fuel Column

In [336]:
cars["fuel"].value_counts()


fuel
Petrol                   6245
Diesel                   3871
Petrol Hybrid              84
Petrol Plug-in Hybrid      19
1 owner                    15
2 owners                    8
4 owners                    7
3 owners                    6
Diesel Hybrid               6
6 owners                    4
Full service history        4
Bi Fuel                     3
Part service history        2
9 owners                    1
11 owners                   1
Name: count, dtype: int64

In [337]:
# cars_comp.shape 
# remove row if doesnt contain "petrol", "disel" or "fuel" in fuel column
cars = cars[cars["fuel"].str.contains("petrol|diesel|fuel", case=False, na=False)]

In [338]:
cars["fuel"].value_counts()


fuel
Petrol                   6245
Diesel                   3871
Petrol Hybrid              84
Petrol Plug-in Hybrid      19
Diesel Hybrid               6
Bi Fuel                     3
Name: count, dtype: int64

Transmission Column

In [339]:
#Column is already clean
cars["transmission"].value_counts()

transmission
Manual       8547
Automatic    1681
Name: count, dtype: int64

In [ ]:
#Creating a column that denotes the make of a car.
cars["brand"] = cars["name"].str.split(" ", expand=True)[0]
cars

In [ ]:
cars["location"] = cars["location"].str.split("location").str[1].str.strip()

In [ ]:
# cars.head()
cars["seller_name"] = cars["seller_name"].str.split("-").str[0].str.strip()

In [343]:
cars.head(3)


,name,year,owners,mileage,litres,BHP,transmission,fuel,price,write_off_category,vehicle_type,seller_name,location,plate,brand
0,KIA Sportage,2018,NaN,23600.0,1.6L,174.0,Manual,Petrol,16990,GC,SUV,Belfast Audi,Belfast,,KIA
1,Vauxhall Astra,2007,Part service history,119000.0,1.9L,118.0,Automatic,Diesel,995,GC,Hatchback,Burch Autos,Newtownards,57,Vauxhall
2,Citroen DS3,2012,NaN,75995.0,1.6L,156.0,Manual,Petrol,1493,GC,Hatchback,Auto Sales NI,Newtownabbey,,Citroen


Owners

In [344]:
cars["owners"].value_counts()


owners
2 owners                   1751
3 owners                   1458
1 owner                     926
4 owners                    902
5 owners                    486
Full service history        367
6 owners                    234
Part service history        227
7 owners                    118
8 owners                     68
9 owners                     36
Full dealership history      25
No service history           25
10 owners                    22
12 owners                    13
11 owners                    11
15 owners                     4
Full Service History          4
14 owners                     3
13 owners                     3
17 owners                     1
Name: count, dtype: int64

In [205]:

cars["owners"].value_counts()


owners
2 owners                      1751
3 owners                      1458
1 owner                        926
4 owners                       902
5 owners                       486
Full service history           371
6 owners                       234
Part service history           232
7 owners                       118
8 owners                        68
9 owners                        36
Full dealership history         27
No service history              26
10 owners                       22
12 owners                       13
11 owners                       11
15 owners                        4
Full Service History             4
14 owners                        3
13 owners                        3
Full manufacturer warranty       2
17 owners                        1
Part Service History             1
Name: count, dtype: int64

In [345]:
cars_comp = cars.drop("owners", axis=1)

In [348]:
cars_comp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10228 entries, 0 to 29366
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                10228 non-null  object 
 1   year                10228 non-null  int32  
 2   mileage             10228 non-null  float64
 3   litres              10228 non-null  object 
 4   BHP                 10228 non-null  float64
 5   transmission        10228 non-null  object 
 6   fuel                10228 non-null  object 
 7   price               10228 non-null  object 
 8   write_off_category  10228 non-null  object 
 9   vehicle_type        10228 non-null  object 
 10  seller_name         9533 non-null   object 
 11  location            8934 non-null   object 
 12  plate               10228 non-null  object 
 13  brand               10228 non-null  object 
dtypes: float64(2), int32(1), object(11)
memory usage: 1.1+ MB


In [350]:
cars_own = cars
#remove if first letter is not a number 
# cars_own = cars_own[cars_own['owners'].str[0].apply(str.isnumeric)]
cars_own['owners'] = cars_own['owners'].fillna('')

cars_own = cars_own[cars_own['owners'].str.match(r'^\d')]



C:\Users\Isaac\AppData\Local\Temp\ipykernel_27884\1221624488.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_own['owners'] = cars_own['owners'].fillna('')


In [351]:
cars_own["owners"].value_counts()   

owners
2 owners     1751
3 owners     1458
1 owner       926
4 owners      902
5 owners      486
6 owners      234
7 owners      118
8 owners       68
9 owners       36
10 owners      22
12 owners      13
11 owners      11
15 owners       4
14 owners       3
13 owners       3
17 owners       1
Name: count, dtype: int64

In [352]:
cars_own["owners"] = cars_own["owners"].str[0].astype(int)

C:\Users\Isaac\AppData\Local\Temp\ipykernel_27884\1621429340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_own["owners"] = cars_own["owners"].str[0].astype(int)


Exporting files to csv

In [353]:

cars_own.to_csv("cars_own2.csv")

In [349]:
cars_comp.to_csv("cars_comp.csv")